In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import collections
import torchvision
import torchvision.transforms as transforms

from PIL import Image
from pathlib import Path
from catalyst.data.augmentor import Augmentor
from catalyst.utils.factory import UtilsFactory
from catalyst.models.segmentation import UNet

from varian.models.segnet import SegNet

In [3]:
np.random.seed(1488)
n_images = 500
bs = 4
n_workers = 4

In [4]:
images = [np.random.uniform(high=256, size=(512, 512)).astype(np.uint8)
          for x in range(n_images)]
masks = [(np.random.uniform(high=1, size=(512, 512)) > 0.5).astype(float)
         for x in range(n_images)]
data = list(zip(images, masks))

train_data = data[:n_images // 2]
valid_data = data[n_images // 2:]

In [5]:
# # Saving mock data
# data_dir = Path('..') / 'data'

# np.save(data_dir / 'mock_data_images', np.stack(images))
# np.save(data_dir / 'mock_data_masks', np.stack(masks))

# (data_dir / 'mock_data_images').mkdir(exist_ok=True)
# (data_dir / 'mock_data_images' / 'train').mkdir(exist_ok=True)
# (data_dir / 'mock_data_images' / 'valid').mkdir(exist_ok=True)

# (data_dir / 'mock_data_masks').mkdir(exist_ok=True)
# (data_dir / 'mock_data_masks' / 'train').mkdir(exist_ok=True)
# (data_dir / 'mock_data_masks' / 'valid').mkdir(exist_ok=True)

# for i, (image, mask) in enumerate(train_data):
#     Image.fromarray(image).save(data_dir / 'mock_data_images' / 'train' / (str(i) + '.tiff'))
#     Image.fromarray(mask).save(data_dir / 'mock_data_masks' / 'train' / (str(i) + '.tiff'))
    
# for i, (image, mask) in enumerate(valid_data):
#     Image.fromarray(image).save(data_dir / 'mock_data_images' / 'valid' / (str(i) + '.tiff'))
#     Image.fromarray(mask).save(data_dir / 'mock_data_masks' / 'valid' / (str(i) + '.tiff'))

In [17]:
data_transform = transforms.Compose([
    # TODO specify augmentations (e.g. histogram normalization)
    Augmentor(
        dict_key="features",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy().astype(np.float32) / 256.).unsqueeze_(0).float()),
    Augmentor(
        dict_key="features",
        augment_fn=lambda x: np.vstack([x, x, x])),
    Augmentor(
        dict_key="targets",
        augment_fn=lambda x: \
            torch.from_numpy(x.copy()).unsqueeze_(0).float()),
])

open_fn = lambda x: {"features": x[0], "targets": x[1]}


train_loader = UtilsFactory.create_loader(
    train_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    workers=n_workers, 
    shuffle=True)

valid_loader = UtilsFactory.create_loader(
    valid_data, 
    open_fn=open_fn, 
    dict_transform=data_transform, 
    batch_size=bs, 
    workers=n_workers, 
    shuffle=False)

loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = valid_loader

In [18]:
# model = UNet(in_channels=1)
model = SegNet(num_classes=1)

In [19]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# scheduler = None  # for OneCycle usage
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 20, 40], gamma=0.3)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)

In [20]:
from catalyst.dl.callbacks import (
    ClassificationLossCallback, 
    BaseMetrics, Logger, TensorboardLogger,
    OptimizerCallback, SchedulerCallback, CheckpointCallback, 
    PrecisionCallback, OneCycleLR)

n_epochs = 50
logdir = "./logs/segmentation_notebook"

callbacks = collections.OrderedDict()

callbacks["loss"] = ClassificationLossCallback()
callbacks["optimizer"] = OptimizerCallback()
callbacks["metrics"] = BaseMetrics()

# OneCylce custom scheduler callback
callbacks["scheduler"] = OneCycleLR(
    cycle_len=n_epochs,
    div=3, cut_div=4, momentum_range=(0.95, 0.85))

# Pytorch scheduler callback
# callbacks["scheduler"] = SchedulerCallback(
#     reduce_metric="loss_main")

callbacks["saver"] = CheckpointCallback()
callbacks["logger"] = Logger()
callbacks["tflogger"] = TensorboardLogger()

In [ ]:
from catalyst.dl.runner import ClassificationRunner

runner = ClassificationRunner(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler)
runner.train_stage(
    loaders=loaders, 
    callbacks=callbacks, 
    logdir=logdir,
    epochs=n_epochs, verbose=True)



0 * Epoch (train):   0% 0/63 [00:00<?, ?it/s]

SegNet(
  (enc1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (enc3): Sequential

/home/ecohen/projects/miniconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


0 * Epoch (train):   0% 0/63 [00:13<?, ?it/s, batch time=13.77238, data time=0.35612, loss_main=0.80295, lr_main=0.00033, momentum_main=0.94987, sample per second=0.29044]

0 * Epoch (train):   2% 1/63 [00:13<14:17, 13.83s/it, batch time=13.77238, data time=0.35612, loss_main=0.80295, lr_main=0.00033, momentum_main=0.94987, sample per second=0.29044]

0 * Epoch (train):   2% 1/63 [00:14<14:48, 14.33s/it, batch time=0.35206, data time=0.00704, loss_main=0.77321, lr_main=0.00033, momentum_main=0.94975, sample per second=11.36175]

0 * Epoch (train):   3% 2/63 [00:14<07:17,  7.16s/it, batch time=0.35206, data time=0.00704, loss_main=0.77321, lr_main=0.00033, momentum_main=0.94975, sample per second=11.36175]

0 * Epoch (train)

0 * Epoch (train):  68% 43/63 [00:35<00:16,  1.22it/s, batch time=0.36390, data time=0.00563, loss_main=0.70088, lr_main=0.00037, momentum_main=0.94454, sample per second=10.99195]

0 * Epoch (train):  68% 43/63 [00:35<00:16,  1.20it/s, batch time=0.36204, data time=0.00524, loss_main=0.70167, lr_main=0.00037, momentum_main=0.94441, sample per second=11.04855]

0 * Epoch (train):  70% 44/63 [00:35<00:15,  1.23it/s, batch time=0.36204, data time=0.00524, loss_main=0.70167, lr_main=0.00037, momentum_main=0.94441, sample per second=11.04855]

0 * Epoch (train):  70% 44/63 [00:36<00:15,  1.21it/s, batch time=0.36453, data time=0.00592, loss_main=0.70138, lr_main=0.00037, momentum_main=0.94428, sample per second=10.97312]

0 * Epoch (train):  71% 45/63 [00:36<00:14,  1.24it/s, batch time=0.36453, data time=0.00592, loss_main=0.70138, lr_main=0.00037, momentum_main=0.94428, sample per second=10.97312]

0 * Epoch (train):  71% 45/63 [00:36<00:14,  1.22it/s, batch time=0.36212, data time=0.005

0 * Epoch (valid):  37% 23/63 [00:05<00:08,  4.55it/s, batch time=0.19677, data time=0.00505, loss_main=8.00667, lr_main=0.00000, momentum_main=0.00000, sample per second=20.32841]

0 * Epoch (valid):  37% 23/63 [00:05<00:09,  4.38it/s, batch time=0.19645, data time=0.00522, loss_main=7.74840, lr_main=0.00000, momentum_main=0.00000, sample per second=20.36181]

0 * Epoch (valid):  38% 24/63 [00:05<00:08,  4.57it/s, batch time=0.19645, data time=0.00522, loss_main=7.74840, lr_main=0.00000, momentum_main=0.00000, sample per second=20.36181]

0 * Epoch (valid):  38% 24/63 [00:05<00:08,  4.41it/s, batch time=0.19632, data time=0.00543, loss_main=7.34534, lr_main=0.00000, momentum_main=0.00000, sample per second=20.37516]

0 * Epoch (valid):  40% 25/63 [00:05<00:08,  4.59it/s, batch time=0.19632, data time=0.00543, loss_main=7.34534, lr_main=0.00000, momentum_main=0.00000, sample per second=20.37516]

0 * Epoch (valid):  40% 25/63 [00:05<00:08,  4.43it/s, batch time=0.19536, data time=0.005

...

3 * Epoch (train):  35% 22/63 [00:12<00:22,  1.83it/s, batch time=0.37864, data time=0.00619, loss_main=0.69709, lr_main=0.00051, momentum_main=0.92281, sample per second=10.56423]

3 * Epoch (train):  35% 22/63 [00:12<00:23,  1.75it/s, batch time=0.37506, data time=0.00545, loss_main=0.69674, lr_main=0.00051, momentum_main=0.92268, sample per second=10.66510]

3 * Epoch (train):  37% 23/63 [00:12<00:21,  1.83it/s, batch time=0.37506, data time=0.00545, loss_main=0.69674, lr_main=0.00051, momentum_main=0.92268, sample per second=10.66510]

3 * Epoch (train):  37% 23/63 [00:13<00:22,  1.76it/s, batch time=0.38331, data time=0.00546, loss_main=0.69640, lr_main=0.00052, momentum_main=0.92255, sample per second=10.43550]

3 * Epoch (train):  38% 24/63 [00:13<00:21,  1.83it/s, batch time=0.38331, data time=0.00546, loss_main=0.69640, lr_main=0.00052, momentum_main=0.92255, sample per second=10.43550]

3 * Epoch (train):  38% 24/63 [00:13<00:22,  1.76it/s, batch time=0.37907, data time=0.005

3 * Epoch (valid):   3% 2/63 [00:01<00:30,  1.99it/s, batch time=0.20045, data time=0.00665, loss_main=0.69393, lr_main=0.00000, momentum_main=0.00000, sample per second=19.95549]

3 * Epoch (valid):   3% 2/63 [00:01<00:36,  1.66it/s, batch time=0.20291, data time=0.00610, loss_main=0.69405, lr_main=0.00000, momentum_main=0.00000, sample per second=19.71348]

3 * Epoch (valid):   5% 3/63 [00:01<00:24,  2.48it/s, batch time=0.20291, data time=0.00610, loss_main=0.69405, lr_main=0.00000, momentum_main=0.00000, sample per second=19.71348]

3 * Epoch (valid):   5% 3/63 [00:01<00:28,  2.13it/s, batch time=0.20056, data time=0.00592, loss_main=0.69401, lr_main=0.00000, momentum_main=0.00000, sample per second=19.94429]

3 * Epoch (valid):   6% 4/63 [00:01<00:20,  2.84it/s, batch time=0.20056, data time=0.00592, loss_main=0.69401, lr_main=0.00000, momentum_main=0.00000, sample per second=19.94429]

3 * Epoch (valid):   6% 4/63 [00:01<00:23,  2.49it/s, batch time=0.20107, data time=0.00533, lo

3 * Epoch (valid):  71% 45/63 [00:09<00:03,  4.64it/s, batch time=0.19793, data time=0.00567, loss_main=0.69395, lr_main=0.00000, momentum_main=0.00000, sample per second=20.20877]

3 * Epoch (valid):  71% 45/63 [00:09<00:03,  4.55it/s, batch time=0.19592, data time=0.00537, loss_main=0.69397, lr_main=0.00000, momentum_main=0.00000, sample per second=20.41685]

3 * Epoch (valid):  73% 46/63 [00:09<00:03,  4.65it/s, batch time=0.19592, data time=0.00537, loss_main=0.69397, lr_main=0.00000, momentum_main=0.00000, sample per second=20.41685]

3 * Epoch (valid):  73% 46/63 [00:10<00:03,  4.56it/s, batch time=0.19993, data time=0.00477, loss_main=0.69420, lr_main=0.00000, momentum_main=0.00000, sample per second=20.00732]

3 * Epoch (valid):  75% 47/63 [00:10<00:03,  4.66it/s, batch time=0.19993, data time=0.00477, loss_main=0.69420, lr_main=0.00000, momentum_main=0.00000, sample per second=20.00732]

3 * Epoch (valid):  75% 47/63 [00:10<00:03,  4.57it/s, batch time=0.19897, data time=0.005

4 * Epoch (train):  35% 22/63 [00:12<00:22,  1.79it/s, batch time=0.38891, data time=0.00524, loss_main=0.69649, lr_main=0.00057, momentum_main=0.91468, sample per second=10.28526]

4 * Epoch (train):  35% 22/63 [00:12<00:23,  1.71it/s, batch time=0.38484, data time=0.00563, loss_main=0.69605, lr_main=0.00057, momentum_main=0.91455, sample per second=10.39382]

4 * Epoch (train):  37% 23/63 [00:12<00:22,  1.79it/s, batch time=0.38484, data time=0.00563, loss_main=0.69605, lr_main=0.00057, momentum_main=0.91455, sample per second=10.39382]

4 * Epoch (train):  37% 23/63 [00:13<00:23,  1.72it/s, batch time=0.38635, data time=0.00534, loss_main=0.69599, lr_main=0.00057, momentum_main=0.91442, sample per second=10.35334]

4 * Epoch (train):  38% 24/63 [00:13<00:21,  1.79it/s, batch time=0.38635, data time=0.00534, loss_main=0.69599, lr_main=0.00057, momentum_main=0.91442, sample per second=10.35334]

4 * Epoch (train):  38% 24/63 [00:13<00:22,  1.72it/s, batch time=0.38941, data time=0.005

4 * Epoch (valid):   3% 2/63 [00:00<00:27,  2.18it/s, batch time=0.20159, data time=0.00682, loss_main=0.69340, lr_main=0.00000, momentum_main=0.00000, sample per second=19.84204]

4 * Epoch (valid):   3% 2/63 [00:01<00:33,  1.80it/s, batch time=0.19623, data time=0.00709, loss_main=0.69344, lr_main=0.00000, momentum_main=0.00000, sample per second=20.38373]

4 * Epoch (valid):   5% 3/63 [00:01<00:22,  2.69it/s, batch time=0.19623, data time=0.00709, loss_main=0.69344, lr_main=0.00000, momentum_main=0.00000, sample per second=20.38373]

4 * Epoch (valid):   5% 3/63 [00:01<00:26,  2.29it/s, batch time=0.19954, data time=0.00609, loss_main=0.69346, lr_main=0.00000, momentum_main=0.00000, sample per second=20.04607]

4 * Epoch (valid):   6% 4/63 [00:01<00:19,  3.05it/s, batch time=0.19954, data time=0.00609, loss_main=0.69346, lr_main=0.00000, momentum_main=0.00000, sample per second=20.04607]

4 * Epoch (valid):   6% 4/63 [00:01<00:22,  2.66it/s, batch time=0.19517, data time=0.00567, lo